# Test your api connection over here

In [ ]:
message = "MACD: 10 \nConsider buying now"
url = f"https://api.telegram.org/bot[BOT_API_KEY]/sendMessage?chat_id=[MY_CHANNEL_NAME]&text={message}"
requests.post(url)

<Response [200]>

# 1. Install and import necessary files for webscrape.

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

# 2. Intialise chrome with the necessary options.


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
exceptions = (NoSuchElementException, StaleElementReferenceException)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# Login function to login into your Trading View account.

In [ ]:
def login():
    global chrome
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome = webdriver.Chrome('chromedriver',options=chrome_options)
    exceptions = (NoSuchElementException, StaleElementReferenceException)
    chrome.get("https://www.tradingview.com/#signin")
    chrome.maximize_window()
    wait = WebDriverWait(chrome, 20)
    wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Email']"))).click()

    # Input your tradingview username and password.
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[name='username']"))).send_keys("INPUT USERNAME")
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[name='password']"))).send_keys("INPUT PASSWORD")

    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[contains(., 'Sign in')]]"))).click()
    chrome.get("https://www.tradingview.com/chart")
    
    # For now you can only select from default crypto such as BTCUSD, ETHUSD as it is accessing it from a different browser.
    wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='ETHUSD']"))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@id='header-toolbar-indicators']/div[1]"))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, "//input[contains(@class, 'input-')]"))).send_keys("macd")
    wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@data-id, 'STD;MACD')]/div[1]"))).click()

    # You may change to your specific time to use the MACD indicator.
    chrome.find_element(by=By.TAG_NAME, value='body').send_keys('4h')
    actions = ActionChains(chrome)
    actions.send_keys(Keys.ENTER)
    actions.perform()

    '''
      You may return the html layout of the page using the code below 
    '''
    # html_source_code = chrome.execute_script("return document.body.innerHTML;")
    # html_soup: BeautifulSoup = BeautifulSoup(html_source_code, 'html.parser')
    # print(html_soup)
    # print(html_source_code)

# To check if chrome is running or not.

In [ ]:
def isBrowserAlive(chrome):
   try:
      chrome.current_url
      # or driver.title
      return True
   except:
      return False

# Get up & down trend of MACD.

In [ ]:
def get_trend(value1, value2): 
  # value1 as MACD & value2 as Signal
  global down_count
  global up_count
  global reset_count
  # Trigger value of 0.5
  trigger = 0.5
  # If Signal value is more than MACD value, increment down_count by 1
  if value1 < value2:
    down_count += 1
    print("down trend:" + str(down_count))
    # Restart up_count to 0 once down_count equals to 1
    # Count resets so to get a strong trend
    if down_count == 1:
      up_count = 0
  # If Signal value is more than MACD value, increment up_count by 1
  if value1 > value2:
    up_count += 1
    print("up trend:" + str(up_count))
    # Restart down_count to 0 once up_count equals to 1
    if up_count == 1:
      down_count = 0
  # If (MACD - Signal) value has a difference below 0.5, increment reset_count by 1 and restart reset_count to 0 if it is more than 4
  if abs(value1 - value2) <= trigger:
    reset_count += 1
    print("reset count:" + str(reset_count))
    if reset_count > 4:
      reset_count = 0

# Broadcast your signal messages to your telegram channel

In [ ]:
def send_results(value1, value2, d_count, u_count, r_count):
  # Trend count (can adjust count to suit your target)
  count = 20
  # High trigger value of 0.5
  h_trig = 0.5
  # Low trigger value of 0.2
  l_trig = 0.2
  # Buy signal will be triggered if (MACD - Signal) value is lower than low trigger value and Signal value more than MACD value.
  if (abs(value1 - value2) <= l_trig and value1 < value2) or 
    (abs(value1 - value2) <= h_trig and value1 < value2 and d_count >= count and r_count == 1):
    message = f"MACD: {value1} Signal: {value2} \nConsider buying now"
    url = f"https://api.telegram.org/bot[BOT_API_KEY]/sendMessage?chat_id=[MY_CHANNEL_NAME]&text={MY_MESSAGE_TEXT}"
    requests.post(url)
  else:
    message = f"MACD: {value1} Signal: {value2} \nConsider selling now"
    url = f"https://api.telegram.org/bot[BOT_API_KEY]/sendMessage?chat_id=[MY_CHANNEL_NAME]&text={MY_MESSAGE_TEXT}"
    requests.post(url)

# Retrieve MACD & Signal values function
- Webscrape both MACD & Signal values from Tradingview site.
- Pass MACD & Signal values to get_trend function and send_results function. 

In [ ]:
def retrieve_macd():
  chrome.switch_to.window(chrome.window_handles[0])
  chrome.maximize_window()
  elements = chrome.find_elements(by=By.XPATH, value="//div[contains(@class, 'valueValue-')]")
  macd = elements[12].get_attribute('innerHTML')
  signal = elements[13].get_attribute('innerHTML')
  
  # Convert to a negative symbol if not error will appear. 
  get_trend(float(macd.replace('−', '-')), float(signal.replace('−', '-')))
  send_results(float(macd.replace('−', '-')), float(signal.replace('−', '-')), down_count, up_count, reset_count)
  print("Macd:"+ macd + " " + "Signal:"+ signal)

# Loop function to run the entire webscrape program

In [ ]:
def loop():
  global down_count
  global up_count
  global reset_count
  down_count = 0
  up_count = 0
  reset_count = 0
  login()
  alive = isBrowserAlive(chrome)
  time.sleep(20)
  while alive:
    retrieve_macd()
    time.sleep(180)

# 3. Initialise the loop function and the program will run.

In [ ]:
loop()

up trend:1
Macd:−17.62 Signal:−18.22
up trend:2
reset count:1
Macd:−17.86 Signal:−18.27
up trend:3
reset count:2
Macd:−18.12 Signal:−18.32
up trend:4
reset count:3
Macd:−18.00 Signal:−18.30
up trend:5
Macd:−17.15 Signal:−18.13
up trend:6
Macd:−16.62 Signal:−18.02
up trend:7
Macd:−16.52 Signal:−18.01
up trend:8
Macd:−16.96 Signal:−18.09
up trend:9
Macd:−17.52 Signal:−18.20
up trend:10
Macd:−17.54 Signal:−18.21
up trend:11
reset count:4
Macd:−17.99 Signal:−18.30
up trend:12
Macd:−17.58 Signal:−18.22
up trend:13
Macd:−17.60 Signal:−18.22
up trend:14
Macd:−17.49 Signal:−18.20
up trend:15
Macd:−16.69 Signal:−18.04
up trend:16
Macd:−16.78 Signal:−18.06
up trend:17
Macd:−16.17 Signal:−17.94
up trend:18
Macd:−16.38 Signal:−17.98
up trend:19
Macd:−17.31 Signal:−18.16
up trend:20
Macd:−17.57 Signal:−18.21
up trend:21
Macd:−17.56 Signal:−18.21
up trend:22
Macd:−17.50 Signal:−18.20
up trend:23
reset count:5
Macd:−17.99 Signal:−18.30
down trend:1
reset count:1
Macd:−18.60 Signal:−18.42
down trend:2

In [ ]:
chrome.quit()